In [3]:
!wget http://www.robots.ox.ac.uk/~vgg/data/flowers/102/102flowers.tgz && mkdir -p ~/data/flowers
!tar -xzf 102flowers.tgz -C ~/data/flowers/
!rm 102flowers.tgz
!wget http://www.robots.ox.ac.uk/~vgg/data/flowers/102/setid.mat
!wget http://www.robots.ox.ac.uk/~vgg/data/flowers/102/imagelabels.mat
!mv setid.mat ~/data/flowers/setid.mat
!mv imagelabels.mat ~/data/flowers/imagelabels.mat

--2020-03-20 06:22:57--  http://www.robots.ox.ac.uk/~vgg/data/flowers/102/102flowers.tgz
Resolving www.robots.ox.ac.uk (www.robots.ox.ac.uk)... 129.67.94.2
Connecting to www.robots.ox.ac.uk (www.robots.ox.ac.uk)|129.67.94.2|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 344862509 (329M) [application/x-gzip]
Saving to: ‘102flowers.tgz’

102flowers.tgz      100%[===================>] 328.89M  87.3MB/s    in 3.8s    

2020-03-20 06:23:01 (86.0 MB/s) - ‘102flowers.tgz’ saved [344862509/344862509]

--2020-03-20 06:23:15--  http://www.robots.ox.ac.uk/~vgg/data/flowers/102/setid.mat
Resolving www.robots.ox.ac.uk (www.robots.ox.ac.uk)... 129.67.94.2
Connecting to www.robots.ox.ac.uk (www.robots.ox.ac.uk)|129.67.94.2|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 14989 (15K)
Saving to: ‘setid.mat’

setid.mat           100%[===================>]  14.64K  --.-KB/s    in 0s      

2020-03-20 06:23:15 (323 MB/s) - ‘setid.mat’ saved [14989/14989]



In [4]:
pip install tensorflow==1.14.0

     |████████████████████████████████| 109.2MB 28kB/s 
     |████████████████████████████████| 3.2MB 28.9MB/s 
     |████████████████████████████████| 491kB 42.8MB/s 
ERROR: tensorflow-federated 0.12.0 has requirement tensorflow~=2.1.0, but you'll have tensorflow 1.14.0 which is incompatible.
ERROR: tensorflow-federated 0.12.0 has requirement tensorflow-addons~=0.7.0, but you'll have tensorflow-addons 0.8.3 which is incompatible.
  Found existing installation: tensorboard 1.15.0
    Uninstalling tensorboard-1.15.0:
      Successfully uninstalled tensorboard-1.15.0
  Found existing installation: tensorflow-estimator 1.15.1
    Uninstalling tensorflow-estimator-1.15.1:
      Successfully uninstalled tensorflow-estimator-1.15.1
  Found existing installation: tensorflow 1.15.0
    Uninstalling tensorflow-1.15.0:
      Successfully uninstalled tensorflow-1.15.0


In [5]:
!ls ~/data/flowers

imagelabels.mat  jpg  setid.mat


In [6]:
cd ~/data/flowers/

/root/data/flowers


In [7]:
ls

imagelabels.mat  jpg/  setid.mat


In [0]:
from scipy.io import loadmat
import warnings
warnings.filterwarnings('ignore')

mat_labels =  loadmat('imagelabels.mat')
label = mat_labels['labels']
label = label[0]

In [9]:
len(label)

8189

In [0]:
for i in range(len(label)):
  label[i] = label[i] - 1

In [11]:
label

array([76, 76, 76, ..., 61, 61, 61], dtype=uint8)

In [12]:
idmat = loadmat('setid.mat')
idmat.keys()
train_num = idmat['trnid']
test_num = idmat['tstid']
valid_num = idmat['valid']
train_num = train_num[0]
test_num = test_num[0]
valid_num = valid_num[0]
print(len(train_num))
print(len(test_num))
print(len(valid_num))

1020
6149
1020


In [13]:
print(train_num)
print(test_num)
print(valid_num)

[6765 6755 6768 ... 8026 8036 8041]
[6734 6735 6737 ... 8044 8045 8047]
[6773 6767 6739 ... 8028 8008 8030]


In [0]:
# Creating X_train, X_test, X_valid sets

import numpy as np
import cv2

train = []
test = []
valid = []

for i in range(len(train_num)):
  dir = 'jpg/image' + '_' + str(train_num[i]).zfill(5)+ '.jpg'  # To read images to image folder
  image = cv2.imread(dir)
  new_image = cv2.resize(image, (128, 128))
  new_image = new_image/255.0
  train.append(new_image)
X_train = np.asarray(train, dtype = np.float32)
# x_train.reshape(256,256,1)

for i in range(len(test_num)):
  dir = 'jpg/image' + '_' + str(test_num[i]).zfill(5) + '.jpg'
  image = cv2.imread(dir)
  new_image = cv2.resize(image, (128, 128))
  new_image = new_image/255.0
  test.append(new_image)
X_test = np.asarray(test, dtype = np.float32)

for i in range(len(valid_num)):
  dir = 'jpg/image' + '_' + str(valid_num[i]).zfill(5) + '.jpg'
  image = cv2.imread(dir)
  new_image  = cv2.resize(image, (128, 128))
  new_image = new_image/255.0
X_valid = np.asarray(valid, dtype=np.float32)
# valid.reshape(256,256,1)

In [15]:
# Creating y_train, y_test and y_valid sets
train_label = []
for i in range(len(train_num)):
  train_label.append(label[train_num[i]-1])
train_label = np.asarray(train_label, dtype = np.float32)

test_label = []
for i in range(len(test_num)):
  test_label.append(label[test_num[i] - 1])
test_label = np.asarray(test_label, dtype = np.float32)

valid_label = []
for i in range(len(valid_num)):
  valid_label.append(label[valid_num[i] - 1])
valid_label = np.asarray(valid_label, dtype = np.float32)

from keras.utils import to_categorical 
y_train = to_categorical(train_label)  # train_label
y_test = to_categorical(test_label)  # test_label
y_valid = to_categorical(valid_label)  # valid_label

Using TensorFlow backend.


In [16]:
print(len(train))
print(len(test))

1020
6149


In [17]:
from keras.models import Sequential 
from keras.layers import Convolution2D
from keras.layers import MaxPooling2D
from keras.layers import Flatten
from keras.layers import Dense

classifier = Sequential() 

In [18]:
# Step 1 - Convolution
classifier.add(Convolution2D(64, 3, 3, input_shape=(128, 128, 3), activation = 'relu'))  # No. of filter maps

In [19]:
# Step 2 - Pooling
classifier.add(MaxPooling2D(pool_size = (2,2)))  # It will reduce the size of the feature maps (it will devide it by 2)

In [0]:
# Adding a second convolutional layer
classifier.add(Convolution2D(64, 3, 3, activation = 'relu')) # no. of featuer detectors, dimensions of feature detectors and activation function
classifier.add(MaxPooling2D(pool_size=(2,2)))  # Will reduce the size of feature maps

In [0]:
# Step 3 - Flattening
classifier.add(Flatten())  # Contains the information of the spatial structure

In [0]:
# Step 4 - Fully connected layer
# Full Connection 
# Hidden Layer
classifier.add(Dense(output_dim = 128, activation = 'relu'))  # Rectifier activation function (adds hidden layer to cnn)
# Output Layer
classifier.add(Dense(output_dim = 102, activation = 'softmax', name="Output"))  # Sigmoid Activation function

In [23]:
# Compiling the CNN
classifier.compile(optimizer = 'adam', loss = 'categorical_crossentropy', metrics = ['accuracy'])

In [0]:
# Making predictions
classifier.fit(X_test, y_test, validation_data = (X_train, y_train), epochs=40)

Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where

Train on 6149 samples, validate on 1020 samples
Epoch 1/40
6149/6149 [==============================] - 266s 43ms/step - loss: 3.9115 - acc: 0.1228 - val_loss: 3.7968 - val_acc: 0.1265
Epoch 2/40
6112/6149 [============================>.] - ETA: 1s - loss: 2.6697 - acc: 0.3233

In [0]:
from google.colab import drive
drive.mount('/content/gdrive')

In [0]:
model_save_name = 'classifier_4373per.h5'
path = F"/content/gdrive/My Drive/flower_categories/classifier.h5" 
classifier.save(path)

In [0]:
classifier.evaluate(X_train, y_train)